# Teste e prototipagem de parseador de CSV do formulário

In [1]:
import pandas as pd
import numpy as np
import json
import datetime as dt 
from zlib import crc32

import xavy.test as xx
import xavy.dataframes as xd

import clean as cc

* Criar identificador único dos casos (talvez possa ser o próprio índice)
* Ver se surgem nulls em datas preenchidas quando as padronizamos. Isso indica formatação estranha.

## Functions

In [2]:
def hash_string(string, prefix=''):
    """
    Takes a `string` as input, remove `prefix` from it and turns it into a hash.
    """
    name   = string.replace(prefix, '')
    return crc32(bytes(name, 'utf-8'))

def build_hash_id(df):
    """
    Return an int Series with hashes build from the 
    content in each line in `df`.
    """
    
    return df.astype(str).sum(axis=1).apply(hash_string)


def embed_metadata(data, metadata, data_key="data", meta_key="metadata"):
    """
    Create a new dict with keys `meta_key` and `data_key` pointing
    to values `metadata` and `data`, respectively.
    
    Returns a dict.
    """
    
    all_data = {meta_key: metadata, data_key: data}
    
    return all_data

In [110]:
def get_translator(translation_df, fields_regex, lower=False):
    """
    Create a dict from portuguese to spanish to be used 
    to translate certain text fields.
    
    Parameters
    ----------
    translation_df : DataFrame
        Table with text field names 'campo' and the textual
        content 'texto_pt' (in portuguese) and 'texto_es' 
        in spanish.
    fields_regex : str
        Regular expression representing the text fields to
        be used.
    lower : bool
        Whether to put everything to lower case.
        
    Returns
    -------
    mapper : dict
        A translation from portuguese to spanish for the 
        fields specified by `fields_regex`.
    """
    
    # Select subset of translations:
    sel_df = translation_df.loc[translation_df['campo'].str.contains(fields_regex)].copy()

    # Put to lowercase:
    if lower == True:
        sel_df['texto_pt'] = sel_df['texto_pt'].str.lower()
        sel_df['texto_es'] = sel_df['texto_es'].str.lower()
    
    # Create translation dict:
    mapper = sel_df[['texto_pt', 'texto_es']].set_index('texto_pt').to_dict()['texto_es']
    
    return mapper


def add_translation(case_json, type_map, topic_map, country_map):
    """
    Add entries (in place) to dict `case_json` containing translations for
    the type, topic and country of the usecase.
    
    Parameters
    ----------
    case_json : dict
        Data about one usecase.
    type_map : dict
        Mapping from portuguese to spanish for possible 
        usecase types.
    topic_map : dict
        Mapping from portuguese to spanish for possible 
        usecase types.
    country_map : dict
        Mapping from portuguese to spanish for possible 
        usecase types.
    """
    
    case_json['type_es']      = [type_map[k] for k in case_json['type']]
    case_json['topics_es']    = [topic_map[k] for k in case_json['topics']]
    case_json['countries_es'] = [country_map[k] for k in case_json['countries']]
    
    return case_json

## Translations

In [97]:
# Load translations:
translation_df = pd.read_excel('../docs/design/textos_site_traducao_traduzidos_v02.xlsx', sheet_name='Site')

# Build translators:
tema_map = get_translator(translation_df, r'filtro_tema_\d')
tipo_map = get_translator(translation_df, r'filtro_tipo_\d', lower=True)
pais_map = get_translator(translation_df, r'filtro_pais_\d')

## Carregando os dados

In [22]:
#raw_df = pd.read_csv('../dados/testes/cordata_formulario/20230825081146_dados-abertos.csv', sep=';')
#raw_df = pd.read_csv('../dados/testes/cordata_formulario/20230904055149_dados-abertos.csv', sep=';')
#raw_df = pd.read_csv('../dados/testes/cordata_formulario/20230914064259_dados-abertos.csv', sep=';')
#raw_df = pd.read_csv('../dados/testes/cordata_formulario/20230920091624_dados-abertos.csv', sep=';')
#raw_df = pd.read_csv('../dados/testes/cordata_formulario/20231025093912_dados-abertos.csv', sep=';')
raw_df = pd.read_csv('../dados/testes/cordata_formulario/20231025104350_dados-abertos.csv', sep=';')

## Limpando os dados

In [23]:
# Tags associated to dummy columns:

topic_names = {'topics_agropecuaria': 'Agropecuária',
 'topics_alimentacao': 'Alimentação',
 'topics_administracao_publica': 'Administração Pública',
 'topics_arte_e_cultura': 'Arte e Cultura',
 'topics_atividade_industrial_de_comercio_ou_servicos': 'Atividade industrial, de comércio ou serviços',
 'topics_ciencia_e_tecnologia': 'Ciência e Tecnologia',
 'topics_comunicacoes': 'Comunicações',
 'topics_consumo': 'Consumo',
 'topics_dados_demograficos': 'Dados demográficos',
 'topics_defesa': 'Defesa',
 'topics_direito_e_processual_penal': 'Direito e Processual Penal',
 'topics_direitos_humanos': 'Direitos Humanos',
 'topics_economia': 'Economia',
 'topics_educacao': 'Educação',
 'topics_energia': 'Energia',
 'topics_esporte': 'Esporte',
 'topics_financas_e_orcamento_publico': 'Finanças e Orçamento Público',
 'topics_imoveis_habitacao_e_urbanismo': 'Imóveis, Habitação e Urbanismo',
 'topics_justica_e_direito': 'Justiça e Direito',
 'topics_lazer': 'Lazer',
 'topics_meio_ambiente': 'Meio Ambiente',
 'topics_multimidia': 'Multimídia',
 'topics_politica_partidos_e_eleicoes': 'Política, Partidos e Eleições',
 'topics_previdencia_e_assistencia_social': 'Previdência e Assistência Social',
 'topics_recursos_hidricos': 'Recursos hídricos',
 'topics_recursos_minerais': 'Recursos minerais',
 'topics_redes_sociais': 'Redes Sociais',
 'topics_relacoes_internacionais': 'Relações Internacionais',
 'topics_religiao': 'Religião',
 'topics_saude': 'Saúde',
 'topics_seguranca': 'Segurança',
 'topics_terras': 'Terras',
 'topics_transporte': 'Transporte',
 'topics_transparencia': 'Transparência',
 'topics_trabalho_e_emprego': 'Trabalho e Emprego',
 'topics_turismo': 'Turismo'}

type_names = {'type_artigo': 'artigo científico ou publicação acadêmica',
 'type_materia_jornalistica': 'matéria jornalística',
 'type_aplicativo_plataforma': 'aplicativo ou plataforma',
 'type_painel': 'painel, dashboard ou infográfico',
 'type_conjunto_de_dados': 'conjunto de dados',
 'type_material': 'material didático',
 'type_estudo': 'estudo independente',
 'type_bot': 'bot',
 'type_outro': 'outro'}

In [24]:
max_datasets = 5
id_offset    = 0

cleaned_df = raw_df.copy()
hash_ids = build_hash_id(raw_df)
cleaned_df.insert(0, 'hash_id', hash_ids)

# Standardize used dataset column names:
cc.rename_first_data_cols(cleaned_df)

# Basic cleaning of strings:
str_cols = ['name', 'url', 'description', 'authors', 'countries', 'email', 'tags', 'url_source', 'url_image', 'comment']
for c in str_cols:
    cleaned_df[c] = cleaned_df[c].str.strip()
    
# Basic cleaning of strings associated to datasets:
data_str_cols = ['data_name', 'data_institution', 'data_url']
for i in range(1, max_datasets + 1):
    for c in data_str_cols:
        col = '{}_{}'.format(c, i)
        cleaned_df[col] = cleaned_df[col].str.strip()

# Remove empty URLs (e.g. 'https://'):
url_cols = xd.sel_col_by_regex(cleaned_df, 'url')
cleaned_df.loc[:, url_cols] = cleaned_df.loc[:, url_cols].replace('^h?t?t?p?s?:?/?/?$', None, regex=True)
        
# Split terms by semicolons:
semicolon_cols = ['authors', 'email', 'tags']
for c in semicolon_cols:
    cleaned_df[c] = cc.split_semicolons(cleaned_df[c])
    
# Split terms by colons:
colon_cols = ['countries']
for c in colon_cols:
    cleaned_df[c] = cc.split_semicolons(cleaned_df[c], ',')

# Parse usecase types and topics covered:
assert set(type_names)  == set(xd.sel_col_by_regex(cleaned_df, '^type_'))
assert set(topic_names) == set(xd.sel_col_by_regex(cleaned_df, '^topics_'))
cleaned_df['type']   = cc.options_to_list(cleaned_df, type_names)
cleaned_df['topics'] = cc.options_to_list(cleaned_df, topic_names)

# Parse datasets used by the usecases:
data_cols = xd.sel_col_by_regex(cleaned_df, '^data_')
data_df  = cleaned_df[data_cols]
cleaned_df['datasets'] = data_df.apply(lambda row: cc.data_info_dict_list(cc.series2transposed_df(row), max_datasets), axis=1)

# Change NaN for None:
cleaned_df = cleaned_df.replace({np.nan: None})

# Add processing date:
cleaned_df['record_date'] = dt.datetime.today().strftime('%Y-%m-%d')

# Select relevant columns:
trash_cols = xd.sel_col_by_regex(cleaned_df, r'type_|topics_|data_|continuar|_repeticao')
good_cols  = [col for col in cleaned_df.columns if col not in trash_cols]
cleaned_json = cleaned_df[good_cols].to_dict(orient='records')

In [28]:
# Add metadata (last update):
metadata = {'last_update': dt.datetime.today().strftime('%Y-%m-%d')}
wmeta_json = embed_metadata(cleaned_json, metadata)

In [29]:
# Load previously saved data:
if True:
    with open('../dados/limpos/usecases_current.json', 'r') as f:
        previous_json = json.load(f)

In [32]:
# Join old data with new data:
updated_json = {'metadata': wmeta_json['metadata'], 
                'data': previous_json['data'] + wmeta_json['data']}

In [111]:
translated_data = [add_translation(case_json, tipo_map, tema_map, pais_map) for case_json in updated_json['data']]

In [113]:
# Join old data with new data:
updated_json = {'metadata': updated_json['metadata'], 
                'data': translated_data}

In [117]:
# Save to file:
if False:
    with open('../dados/limpos/usecases_n04_2023-10-25.json', 'w') as f:
        json.dump(updated_json, f, ensure_ascii=False, indent=2)

## Enriching data

In [37]:
institutions = set(cleaned_df[xd.sel_col_by_regex(cleaned_df, 'data_institution_')].values.flatten())
institutions

{'CGU',
 'Câmara dos deputados',
 'Ministério da Economia',
 None,
 'Prefeitura do Município de São Paulo',
 'SEADE',
 'Senado Federal',
 'TSE'}

## Visualizing cleaned data

In [8]:
s_cols = 9
n_cols = cleaned_df.shape[-1]
for i in range(n_cols // s_cols + 1):
    display(cleaned_df.iloc[:, i * s_cols:(i + 1) * s_cols])

,name,url,description,pub_date,authors,countries,email,type_artigo,type_materia_jornalistica
0,Três partidos do Centrão controlam mais de R$ ...,https://oglobo.globo.com/politica/tres-partido...,Matéria jornalística publicada no Jornal O Glo...,01/2022,"[Dimitrius Dantas, Daniel Gullino, Bruno Góes]",[Brasil],"[bruno.goes@oglobo.com.br, daniel.gullino@bsb....",0,1
1,Serenata de amor,https://serenata.ai,Um projeto aberto que usa ciência de dados com...,06/2016,"[Open Knowledge Brasil, Irio Musskopf]",[Brasil],[contato@serenata.ai],0,0
2,Base dos Dados,https://basedosdados.org/,Catálogo de conjuntos de dados com alguns conj...,09/2020,[Instituto Base dos Dados],"[Mundial, Brasil]",NaN,0,0
3,Tutorial de dados,https://github.com/tecMTST/tutorial-de-dados,Pequenos projetos de análise de dados desenvol...,09/2023,[Núcleo de Tecnologia do MTST],[Brasil],[mtst@nucleodetecnologia.com.br],0,0


,type_aplicativo_plataforma,type_painel,type_conjunto_de_dados,type_material,type_estudo,type_bot,type_outro,topics_agropecuaria,topics_alimentacao
0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,1,0,0,0
2,0,0,1,0,0,0,0,1,1
3,0,0,0,1,1,0,0,0,0


,topics_administracao_publica,topics_arte_e_cultura,topics_atividade_industrial_de_comercio_ou_servicos,topics_ciencia_e_tecnologia,topics_comunicacoes,topics_consumo,topics_dados_demograficos,topics_defesa,topics_direito_e_processual_penal
0,1,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1
3,0,0,0,0,0,0,0,0,0


,topics_direitos_humanos,topics_economia,topics_educacao,topics_energia,topics_esporte,topics_financas_e_orcamento_publico,topics_imoveis_habitacao_e_urbanismo,topics_justica_e_direito,topics_lazer
0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,1,0,0,0
2,1,1,1,1,1,1,1,1,1
3,1,0,0,0,0,0,1,0,0


,topics_meio_ambiente,topics_multimidia,topics_politica_partidos_e_eleicoes,topics_previdencia_e_assistencia_social,topics_recursos_hidricos,topics_recursos_minerais,topics_redes_sociais,topics_relacoes_internacionais,topics_religiao
0,0,0,1,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1
3,0,0,1,0,0,0,0,0,0


,topics_saude,topics_seguranca,topics_terras,topics_transporte,topics_transparencia,topics_trabalho_e_emprego,topics_turismo,tags,url_source
0,0,0,0,0,0,0,0,"[orçamento federal, orçamento secreto, bolsona...",https://github.com/gabinete-compartilhado-acre...
1,0,0,0,0,1,0,0,"[ceap, Cota para Exercício da Atividade Parlam...",https://github.com/okfn-brasil/serenata-de-amor
2,1,1,1,1,1,1,1,"[catálogo, repositório]",https://github.com/basedosdados
3,1,0,0,0,0,0,0,"[tutorial, análise de dados, ciência de dados,...",https://github.com/tecMTST/tutorial-de-dados


,url_image,data_name_1,data_institution_1,data_url_1,data_periodical_1,continuar,data_name_2,data_institution_2,data_url_2
0,https://ogimg.infoglobo.com.br/in/25354165-974...,SIGA Brasil,Senado Federal,https://www12.senado.leg.br/orcamento/sigabrasil,2,1,Orçamento da despesa pública,CGU,https://portaldatransparencia.gov.br/orcamento...
1,https://d33wubrfki0l68.cloudfront.net/44a83a26...,Cota parlamentar,Câmara dos deputados,https://www.camara.leg.br/cota-parlamentar/,1,1,Informações sobre deputados federais,Câmara dos deputados,https://www.camara.leg.br/SitCamaraWS/Deputado...
2,https://avatars.githubusercontent.com/u/710976...,NaN,NaN,NaN,1,2,NaN,NaN,NaN
3,https://nucleodetecnologia.com.br/assets/img/n...,Mortalidade no município de Sâo Paulo,Prefeitura do Município de São Paulo,https://www.prefeitura.sp.gov.br/cidade/secret...,2,1,"População do município de São Paulo, por idade",SEADE,https://repositorio.seade.gov.br/group/seade-p...


,data_periodical_2,segunda_repeticao,data_name_3,data_institution_3,data_url_3,data_periodical_3,terceira_repeticao,data_name_4,data_institution_4
0,2.0,1.0,Plataforma +Brasil,Ministério da Economia,https://antigo.plataformamaisbrasil.gov.br/dow...,2.0,2.0,NaN,NaN
1,1.0,1.0,API de Compras Governamentais,Ministério da Economia,http://compras.dados.gov.br,1.0,2.0,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2.0,1.0,Fronteiras dos distritos do município de São P...,Prefeitura do Município de São Paulo,https://geosampa.prefeitura.sp.gov.br,2.0,1.0,Prestação de contas eleitorais,TSE


,data_url_4,data_periodical_4,quarta_repeticao,data_name_5,data_institution_5,data_url_5,data_periodical_5,comment,type
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[matéria jornalística]
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[painel, dashboard ou infográfico, bot]"
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"São muitas bases de dados, não seria possível ...",[conjunto de dados]
3,https://dadosabertos.tse.jus.br/dataset/,2.0,1.0,Candidatos eleitorais,TSE,https://dadosabertos.tse.jus.br/dataset/,2.0,NaN,"[material didático, estudo independente]"


,topics,datasets
0,"[Administração Pública, Finanças e Orçamento P...","[{'data_name': 'SIGA Brasil', 'data_institutio..."
1,"[Administração Pública, Finanças e Orçamento P...","[{'data_name': 'Cota parlamentar', 'data_insti..."
2,"[Agropecuária, Alimentação, Administração Públ...",[]
3,"[Direitos Humanos, Imóveis, Habitação e Urbani...",[{'data_name': 'Mortalidade no município de Sâ...


## Testando funções

In [28]:
good_series = pd.Series(['1970-01', '1970-12', '1970-9', '2099-1', '2099-10', 
                         '01/1970', '12/1970', '9/1970', '1/2099', '10/2099',
                         ' 1970-01', '1970-12 ', ' 1970-9 ', '  2099-1  ', '2099-10', 
                         '01/1970 ', '  12/1970', '9/1970 ', '  1/2099', '10/2099  ', 
                         np.NaN, 'janeiro de 2020', '13/2000', '1800-01', '1900-01'])
good_answers = pd.Series(['1970-01', '1970-12', '1970-09', '2099-01', '2099-10', 
                          '1970-01', '1970-12', '1970-09', '2099-01', '2099-10', 
                          '1970-01', '1970-12', '1970-09', '2099-01', '2099-10', 
                          '1970-01', '1970-12', '1970-09', '2099-01', '2099-10',
                         np.NaN, np.NaN, np.NaN, np.NaN, '1900-01'])

xx.multi_test_function(std_date_series, [(good_series,)], [good_answers])

In [13]:
good_series  = pd.Series(['http://cordata.nic.br', 'https://x.com/v2/api', 'ftp://henriquexavier.net', 'did:google.com?q=abacate&t=39', 'http://www.ftp.f/aba#dsh/fvuih487&#%$@$#.html', np.NaN,
                          'htp://www.globo.com', 'www.gov.br', 'http://www efd.com', 'http://gov.br/index.html ', '   http://gov.br/index.html ', ''])
good_answers = pd.Series([False, False, False, False, False, False, True, True, True, False, False, False])

xx.multi_test_function(bad_url, [(good_series,)], [good_answers])